In [7]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import plotly.express as px

import re
import regex
import emoji
from collections import Counter
from wordcloud import WordCloud, STOPWORDS

# De txt a tabla

## Funciones

In [8]:
def IniciaConFechaYHora(s):
    # Ejemplo: '2/10/2018, 9:38 p. m. - ... '
    patron = '^([1-9]|1[0-9]|2[0-9]|3[0-1])(\/)([1-9]|1[0-2])(\/)(2[0-9][0-9][0-9]), ([0-9]|0[0-9]|1[0-2]):([0-9][0-9])\u202f(p.\u202fm.|a.\u202fm.) -'
    resultado = re.match(patron, s)  # Verificar si cada línea del txt hace match con el patrón de fecha y hora
    if resultado:
        return True
    return False

In [9]:
# Patron para encontrar a los miembros del grupo dentro del txt
def EncontrarMiembro(s):
    patrones = [
        '([\w]+):',                                    # Nombre
        '([\w]+[\s]+[\(]+[\w]+[\)]+):',      # Nombre (Apodo)
        '([\w]+[\s]+[\w]+):',                    # Nombre + Apellido
        '([\w]+[\s]+[\w]+[\s]+[\w]+):',    # Nombre 1 + Nombre 2 + Apellido
        '([+]\d{2} \d{3} \d{3} \d{4}):',     # Número de teléfono (MX)
        '([\w]+)[\u263a-\U0001f999]+:', # Nombre + Emoji            
    ]
    patron = '^' + '|'.join(patrones)     
    resultado = re.match(patron, s)  # Verificar si cada línea del txt hace match con el patrón de miembro
    if resultado:
        return True
    return False

In [10]:
# Separar las partes de cada línea del txt: Fecha, Hora, Miembro y Mensaje
def ObtenerPartes(linea):   
    # Ejemplo: '21/2/2021 11:27 a. m. - Sandro: Todos debemos aprender a analizar datos'
    splitLinea = linea.split(' - ') 
    FechaHora = splitLinea[0]                     # '21/2/2021 11:27 a. m.'
    splitFechaHora = FechaHora.split(' ')   
    Fecha = splitFechaHora[0]                    # '21/2/2021'
    Hora = ' '.join(splitFechaHora[1:])          # '11:27 a. m.'
    Mensaje = ' '.join(splitLinea[1:])             # 'Sandro: Todos debemos aprender a analizar datos'
    if EncontrarMiembro(Mensaje): 
        splitMensaje = Mensaje.split(': ')      
        Miembro = splitMensaje[0]               # 'Sandro' 
        Mensaje = ' '.join(splitMensaje[1:])    # 'Todos debemos aprender a analizar datos'
    else:
        Miembro = None
    return Fecha, Hora, Miembro, Mensaje

### Unit test

In [3]:

import unittest

class TestIniciaConFechaYHora(unittest.TestCase):
    def test_IniciaConFechaYHora(self):
        self.assertTrue(IniciaConFechaYHora("24/10/2018, 10:36 p. m. - Paola Daft: 😹😹😹"))
        self.assertFalse(IniciaConFechaYHora("Esto no comienza con una fecha y hora"))
        self.assertTrue(IniciaConFechaYHora("1/1/2000, 12:00 a. m. - Usuario: mensaje"))
        self.assertFalse(IniciaConFechaYHora("24/10/2018 - Paola Daft: 😹😹😹"))  # Falta la hora
        self.assertFalse(IniciaConFechaYHora("10:36 p. m. - Paola Daft: 😹😹😹"))  # Falta la fecha

suite = unittest.TestLoader().loadTestsFromTestCase(TestIniciaConFechaYHora)
unittest.TextTestRunner().run(suite)

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

## Obtención de las partes del chat

In [16]:
# Leer el archivo txt descargado del chat de WhatsApp
RutaChat = './data/Chat de WhatsApp con Carlitos.txt'

encryption_msg = "Los mensajes y las llamadas están cifrados de extremo a extremo. Nadie fuera de este chat, ni siquiera WhatsApp, puede leerlos ni escucharlos. Toca para obtener más información."


# Lista para almacenar los datos (Fecha, Hora, Miembro, Mensaje) de cada línea del txt 
DatosLista = [] 
with open(RutaChat, encoding="utf-8") as fp:
    fp.readline() # Eliminar primera fila relacionada al cifrado de extremo a extremo
    VerificarMensaje = [] # Lista para verificar que no existan mensajes vacíos 
    Fecha, Hora, Miembro = None, None, None
    while True:
        linea = fp.readline() 
        if not linea: 
            break
        linea = linea.strip() 
        if encryption_msg in linea:
            continue  # Skip this line and move to the next one
        if IniciaConFechaYHora(linea): # Si cada línea del txt coincide con el patrón fecha y hora
            if len(VerificarMensaje) > 0: 
                # Añadir un elemento lista que contendrá los datos a la lista 'DatosLista' 
                DatosLista.append([Fecha, Hora, Miembro, ' '.join(VerificarMensaje)]) 
            VerificarMensaje.clear() 
            Fecha, Hora, Miembro, Mensaje = ObtenerPartes(linea) # Obtener datos de cada línea del txt
            VerificarMensaje.append(Mensaje) 
        else:
            VerificarMensaje.append(linea)

# Convertir la lista con los datos a dataframe
df = pd.DataFrame(DatosLista, columns=['Fecha', 'Hora', 'Miembro', 'Mensaje']) 

# Cambiar la columna Fecha a formato datetime
df['Fecha'] = pd.to_datetime(df['Fecha'], format="%d/%m/%Y,")

# Eliminar los posibles campos vacíos del dataframe
# y lo que no son mensajes como cambiar el asunto del grupo o agregar a alguien
df = df.dropna()

# Rester el índice
df.reset_index(drop=True, inplace=True)
df

,Fecha,Hora,Miembro,Mensaje
0,2018-10-02,9:38 p. m.,Paola Daft,Uffff when sales temprano y no tienes tarea :'v
1,2018-10-02,9:39 p. m.,Carlitos,¿Quien dice que no tengo tarea?
2,2018-10-02,9:40 p. m.,Paola Daft,😹😹😹
3,2018-10-02,9:40 p. m.,Carlitos,Tengo un putero pero siempre hay que despejars...
4,2018-10-02,9:40 p. m.,Paola Daft,When salea temprano y te vale la tarea XD
...,...,...,...,...
7148,2024-02-12,7:12 p. m.,Paola Daft,<Multimedia omitido>
7149,2024-02-12,7:12 p. m.,Paola Daft,<Multimedia omitido>
7150,2024-02-12,7:12 p. m.,Paola Daft,<Multimedia omitido>
7151,2024-02-12,7:12 p. m.,Paola Daft,<Multimedia omitido>


# Estadísticas de mensajes, multimedia, emojis y links

## Total de mensajes, multimedia, emojis y links enviados


def ObtenerEmojis(Mensaje):
    emoji_lista = []
    data = regex.findall(r'\X', Mensaje) # Obtener lista de caracteres de cada mensaje
    for caracter in data:
        if any(c in emoji.UNICODE_EMOJI['es'] for c in caracter): # Obtener emojis en idioma español: 'es'
            emoji_lista.append(caracter)
    return emoji_lista

# Obtener la cantidad total de mensajes
total_mensajes = df.shape[0] 

# Obtener la cantidad de archivos multimedia enviados
multimedia_mensajes = df[df['Mensaje'] == '<Multimedia omitido>'].shape[0] 

# Obtener la cantidad de emojis enviados
df['Emojis'] = df['Mensaje'].apply(ObtenerEmojis) # Se agrega columna 'Emojis'
emojis = sum(df['Emojis'].str.len())

# Obtener la cantidad de links enviados
url_patron = r'(https?://\S+)'
df['URLs'] = df.Mensaje.apply(lambda x: re.findall(url_patron, x)).str.len() # Se agrega columna 'URLs'
links = np.sum(df.URLs)

# Todos los datos pasarlo a diccionario
estadistica_dict = {'Tipo': ['Mensajes', 'Multimedia', 'Emojis', 'Links'],
        'Cantidad': [total_mensajes, multimedia_mensajes, emojis, links]
        }

#Convertir diccionario a dataframe
estadistica_df = pd.DataFrame(estadistica_dict, columns = ['Tipo', 'Cantidad'])

# Establecer la columna Tipo como índice
estadistica_df = estadistica_df.set_index('Tipo')
estadistica_df